In [ ]:
import ants
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

from zebrafishframework import io
from importlib import reload

In [ ]:
io = reload(io)

In [ ]:
LIF_FILE = '/Users/koesterlab/calcium/stimulus/fish46_6dpf_amph.lif'

In [ ]:
ts = [100, 1500]
frames = [io.get_frame(LIF_FILE, t) for t in ts]

In [ ]:
@interact
def browse(i:(0,20)):
    plt.figure(figsize=(12,12))
    plt.imshow(frames[0][i])

In [ ]:
@interact
def browse(i:(0,20)):
    plt.figure(figsize=(12,12))
    plt.imshow(frames[1][i])

In [ ]:
ants.registration?

In [ ]:
frames_ants = [ants.from_numpy(f.swapaxes(0, 2)) for f in frames]

In [ ]:
res = ants.registration(frames_ants[0], frames_ants[1],
                  type_of_transform='Translation',
                  grad_step=.2)

In [ ]:
t = ants.read_transform(res['fwdtransforms'][0])
zshift = t.parameters[-1]
zshift = int(round(zshift))

In [ ]:
warped = np.zeros_like(frames[0])
for z in range(max((0, -zshift)), min((21, -zshift + 21))):
    print(z)
    mov = ants.from_numpy(frames[1][z + zshift].swapaxes(0, 1))
    fix = ants.from_numpy(frames[0][z].swapaxes(0, 1))
    res = ants.registration(mov, fix,
                     type_of_transform='Affine',
                     reg_iterations=[500, 500, 500],
                     grad_step=.1,
                     verbose=True)
    t = ants.read_transform(res['fwdtransforms'][0])
    trans = ants.apply_ants_transform_to_image(t, mov, fix)
    warped[z] = trans.numpy().swapaxes(0, 1)

In [ ]:
@interact
def browse(i:(0,20)):
    plt.figure(figsize=(12,12))
    plt.imshow(warped[i])

In [ ]:
#io.save('/Users/koesterlab/bla/frame0.nrrd', frames[0], spacing=io.SPACING_JAKOB)
#io.save('/Users/koesterlab/bla/frame1.nrrd', frames[1], spacing=io.SPACING_JAKOB)
io.save('/Users/koesterlab/bla/frame1_warped2.nrrd', warped, spacing=io.SPACING_JAKOB)